In [1]:
import pandas as pd
import numpy as np
import pymc as pm
from scipy.stats import binom
import arviz as az
from fancyimpute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
int32 = np.int32
import multiprocessing
from sklearn.model_selection import train_test_split


# read in the police reports and real-time crime center data
pr = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")
rtcc = pd.read_csv("../data/real_time_crime_center/rtcc.csv")
rtcc["rtcc_requested"] = 1 

# create a new column in the police reports data indicating whether the offender is Black
pr['race_black'] = (pr['offender_race'] == 'BLACK')

# select only the necessary columns from the police reports data and join with the distinct item numbers from the rtcc data
rc = pd.merge(pr, rtcc, on="item_number", how="left")

rc['rtcc_requested'] = rc['rtcc_requested'].fillna(0)

rc.loc[:, "rtcc_requested"] = rc.rtcc_requested.astype(str).str.replace(r"\.0", "", regex=True)
rc.loc[:, "offenderid"] = rc.offenderid.astype(str).str.replace(r"\.0", "", regex=True)

rc = rc[~((rc.charge_description.fillna("") == ""))]
rc = rc[~((rc.item_number.fillna("") == ""))]

# Replace empty and 'unknown' strings with NaN
rc['offender_race'] = rc['offender_race'].replace('', np.nan).fillna(value=np.nan, inplace=False)
rc['offender_race'] = rc['offender_race'].replace('UNKNOWN', np.nan)

# Create a new dataframe with only the 'district' and 'offender_race' columns
subset_df = rc[['district', 'offender_race']]

# Split the data into known and unknown values based on the missing values in the 'offender_race' column
known = subset_df[subset_df['offender_race'].notna()]
unknown = subset_df[subset_df['offender_race'].isna()]

# Encode categorical data using LabelEncoder
le = LabelEncoder()
known['district'] = le.fit_transform(known['district'])
known['offender_race'] = le.fit_transform(known['offender_race'])

# Impute missing values using IterativeImputer
imputer = IterativeImputer()
imputed_values = imputer.fit_transform(known)

# Convert imputed values back to dataframe
imputed_df = pd.DataFrame(imputed_values, columns=['district', 'offender_race'])
imputed_df['district'] = imputed_df['district'].astype(int)
imputed_df['offender_race'] = imputed_df['offender_race'].round().astype(int)

# Convert the numerical data back to categorical data using LabelEncoder's inverse_transform method
imputed_df['offender_race'] = le.inverse_transform(imputed_df['offender_race'])

# Replace the missing values in the original 'offender_race' column with the imputed values
rc.loc[unknown.index, 'offender_race'] = imputed_df['offender_race']

df_0 = rc[rc['rtcc_requested'] == "0"]
df_1 = rc[rc['rtcc_requested'] == '1']

charges_rtcc = [x for x in df_1["charge_description"]]

df_0 = df_0[df_0.charge_description.isin(charges_rtcc)]

charges_new_rtcc = [x for x in df_0["charge_description"]]

df_1 = df_1[df_1.charge_description.isin(charges_new_rtcc)]

rc = pd.concat([df_0, df_1])

rc = rc.drop_duplicates(subset=["item_number", "offenderid"])
rc.loc[:, "rtcc_requested"] = rc.rtcc_requested.astype(int)

rc_grouped = rc.groupby(['race_black', 'charge_description']).agg(n=('item_number', 'count'), rtcc=('rtcc_requested', 'sum')).reset_index()
rc_grouped['race_charge'] = pd.Categorical(rc_grouped['race_black'].astype(str) + '_' + rc_grouped['charge_description'])

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (13,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [2]:
# # Split the rc_grouped dataframe into train and test dataframes
# train_data, test_data = train_test_split(rc_grouped, test_size=0.2, random_state=42)

train_data = rc_grouped[rc_grouped.charge_description != "ILLEGAL POSSESSION OF STOLEN THINGS"]
test_data = rc_grouped[rc_grouped.charge_description == "ILLEGAL POSSESSION OF STOLEN THINGS"]

# Convert charge_description to categorical variable
charge_descr_cats_train = train_data['charge_description'].astype('category').cat.categories.values
charge_descr_ints_train = train_data['charge_description'].astype('category').cat.codes.values

with pm.Model() as train_model:
    # Prior for the regression coefficients
    b = pm.Normal('b', mu=0, sigma=1)
    b_race = pm.Normal('b_race', mu=0, sigma=1)
    
    # Priors for the standard deviation of the intercept
    sd_intercept = pm.HalfCauchy('sd_intercept', beta=1)
    
    # Define n as a data input
    n = pm.Data('n', train_data['n'].values, mutable=True)
    
    # Define race_black as a predictor
    race_black = pm.Data('race_black', train_data['race_black'].astype(int).values, mutable=True)
    
    # Model the intercept as a normal distribution with a group-level standard deviation
    intercept = pm.Normal('intercept', mu=0, sigma=sd_intercept, shape=len(charge_descr_cats_train))
    
    # Model the effects of the charge descriptions
    charge_effect = pm.Normal('charge_effect', mu=0, sigma=50, shape=len(charge_descr_cats_train))
    
    # Calculate the linear predictor
    lp = intercept[charge_descr_ints_train] + b * race_black + charge_effect[charge_descr_ints_train]
    
    # Model the counts of RTCC requests as a Poisson distribution
    rtcc_est = pm.Poisson('rtcc_est', mu=pm.math.exp(lp), shape=len(train_data))
    
    # Prior for the standard deviation of the error term
    sigma = pm.HalfCauchy('sigma', beta=1)
    
    # Define the observed RTCC requests
    rtcc = pm.Poisson('rtcc', mu=rtcc_est, observed=train_data['rtcc'].values)

with train_model:
    trace_train = pm.sample(100, tune=100, chains=16, cores=56, target_accept=0.9)

print("Training model complete")

Only 100 samples in chain.
Multiprocess sampling (16 chains in 56 jobs)
CompoundStep
>NUTS: [b, b_race, sd_intercept, intercept, charge_effect, sigma]
>Metropolis: [rtcc_est]


/home/ayyubi/.local/lib/python3.7/site-packages/pymc/step_methods/metropolis.py:289: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/home/ayyubi/.local/lib/python3.7/site-packages/pymc/step_methods/metropolis.py:289: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/home/ayyubi/.local/lib/python3.7/site-packages/pymc/step_methods/metropolis.py:289: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/home/ayyubi/.local/lib/python3.7/site-packages/pymc/step_methods/metropolis.py:289: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/home/ayyubi/.local/lib/python3.7/site-packages/pymc/step_methods/metropolis.py:289: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/home/ayyubi/.local/lib/python3.7/site-packages/pymc/step_methods/metropolis.py:289: RuntimeWar

Training model complete


In [3]:
# Generate predictions for the test dataframe
charge_descr_cats_test = test_data['charge_description'].astype('category').cat.categories.values
charge_descr_ints_test = test_data['charge_description'].astype('category').cat.codes.values
with pm.Model() as test_model:
    # Priors for the regression coefficients
    b = pm.Normal('b', mu=0, sigma=1)
    b_race = pm.Normal('b_race', mu=0, sigma=1)
    
    # Priors for the standard deviation of the intercept
    sd_intercept = pm.HalfCauchy('sd_intercept', beta=1)
    
    # Define n as a data input
    n = pm.Data('n', test_data['n'].values, mutable=True)
    
    # Define race_black as a predictor
    race_black = pm.Data('race_black', test_data['race_black'].astype(int).values, mutable=True)
    
    # Model the intercept as a normal distribution with a group-level standard deviation
    intercept = pm.Normal('intercept', mu=0, sigma=sd_intercept, shape=len(charge_descr_cats_train))
    
    # Model the effects of the charge descriptions
    charge_effect = pm.Normal('charge_effect', mu=0, sigma=50, shape=len(charge_descr_cats_train))
    
    # Calculate the linear predictor
    lp = intercept[charge_descr_ints_test] + b * race_black + charge_effect[charge_descr_ints_test]
    
    # Model the counts of RTCC requests as a Poisson distribution
    rtcc_est = pm.Poisson('rtcc_est', mu=pm.math.exp(lp), shape=len(test_data))
    
    # Prior for the standard deviation of the error term
    sigma = pm.HalfCauchy('sigma', beta=1)
    
    rtcc = pm.Poisson('rtcc', mu=rtcc_est, shape=len(test_data))
    
    # Sample from the posterior distribution
    with test_model:
        trace_test = pm.sample_posterior_predictive(trace_train, var_names=['rtcc_est'])

print("Testing model complete")

Testing model complete


In [7]:
rtcc_est_pred = trace_test.posterior_predictive['rtcc_est']

charge_descr_cats_test = test_data['charge_description'].astype('category').cat.categories.values
charge_descr_ints_test = test_data['charge_description'].astype('category').cat.codes.values

rtcc_est_pred_summary = az.summary(trace_test.posterior_predictive['rtcc_est'])
rtcc_est_pred_summary.index = charge_descr_cats_test[charge_descr_ints_test]

observed_rtcc = test_data['rtcc'].values
race = test_data["race_black"].values
charges = test_data["race_charge"].values
rtcc_est_pred_summary['observed_rtcc'] = observed_rtcc
rtcc_est_pred_summary["race_black"] = race
rtcc_est_pred_summary["charge"] = charges

print(rtcc_est_pred_summary)

                                      mean     sd  hdi_3%  hdi_97%  mcse_mean  \
ILLEGAL POSSESSION OF STOLEN THINGS  4.033  4.445     0.0     12.0      0.806   
ILLEGAL POSSESSION OF STOLEN THINGS  4.871  5.424     0.0     16.0      1.014   

                                     mcse_sd  ess_bulk  ess_tail  r_hat  \
ILLEGAL POSSESSION OF STOLEN THINGS    0.575      30.0      40.0   1.57   
ILLEGAL POSSESSION OF STOLEN THINGS    0.724      28.0      38.0   1.62   

                                     observed_rtcc  race_black  \
ILLEGAL POSSESSION OF STOLEN THINGS             49       False   
ILLEGAL POSSESSION OF STOLEN THINGS            130        True   

                                                                        charge  
ILLEGAL POSSESSION OF STOLEN THINGS  False_ILLEGAL POSSESSION OF STOLEN THINGS  
ILLEGAL POSSESSION OF STOLEN THINGS   True_ILLEGAL POSSESSION OF STOLEN THINGS  


In [5]:
rtcc_est_pred_summary.iloc[:50]

,mean,observed_rtcc,race_black
ILLEGAL POSSESSION OF STOLEN THINGS,4.033,49,False
ILLEGAL POSSESSION OF STOLEN THINGS,4.871,130,True


In [15]:
charge_descr_cats = rc_grouped['charge_description'].astype('category').cat.categories.values
charge_descr_ints = rc_grouped['charge_description'].astype('category').cat.codes.values

with pm.Model() as model:
    # Prior for the regression coefficients
    b = pm.Normal('b', mu=1, sigma=1)
    b_race = pm.Normal('b_race', mu=0, sigma=1)
    
    # Priors for the standard deviation of the intercept
    sd_intercept = pm.HalfCauchy('sd_intercept', beta=5)
    
    # Define n as a data input
    n = pm.Data('n', rc_grouped['n'].values, mutable=True)
    
    # Define race_black as a predictor
    race_black = pm.Data('race_black', rc_grouped['race_black'].astype(int).values, mutable=True)
    
    # Model the intercept as a normal distribution with a group-level standard deviation
    intercept = pm.Normal('intercept', mu=1, sigma=sd_intercept, shape=len(charge_descr_cats))
    
    # Model the effects of the charge descriptions
    charge_effect = pm.Normal('charge_effect', mu=1, sigma=1, shape=len(charge_descr_cats))
    
    # Calculate the linear predictor
    lp = intercept[charge_descr_ints] + b * race_black + charge_effect[charge_descr_ints]
    
    # Model the counts of RTCC requests as a Poisson distribution
    rtcc_est = pm.Poisson('rtcc_est', mu=pm.math.exp(lp), shape=len(rc_grouped))
    
    # Prior for the standard deviation of the error term
    sigma = pm.HalfCauchy('sigma', beta=1)
    
    # Define the observed RTCC requests
    rtcc = pm.Poisson('rtcc', mu=rtcc_est, observed=rc_grouped['rtcc'].values)

with model:
    trace = pm.sample(1000, tune=1000, chains=8, cores=56, target_accept=0.9)

with model:
    trace_post = pm.sample_posterior_predictive(trace, var_names=['charge_effect'])

print("Finished modelling")

Multiprocess sampling (8 chains in 56 jobs)
CompoundStep
>NUTS: [b, b_race, sd_intercept, intercept, charge_effect, sigma]
>Metropolis: [rtcc_est]


Sampling 8 chains for 1_000 tune and 1_000 draw iterations (8_000 + 8_000 draws total) took 346 seconds.


Finished modelling


In [16]:
coef_summary = az.summary(trace_post.posterior_predictive)
coef_summary.index = rc_grouped["charge_description"].unique()

coef_summary = coef_summary[["mean", "hdi_3%", "hdi_97%"]]
coef_summary.columns = ["mean", "q05", "q95"]

# ## take the median?
coef_summary = coef_summary.sort_values("mean", ascending=False)
print(coef_summary.head(20))

                                                     mean    q05    q95
CRIMINAL DAMAGE TO HISTORIC BUILDINGS OR LANDMA...  1.030 -0.852  2.922
POSSESSION OF STOLEN PROPERTY                       1.026 -0.875  2.884
BATTERY OF A DATING PARTNER (CHILD ENDANGERMENT...  1.025 -0.808  2.872
PROPER EQUIPMENT REQUIRED                           1.025 -0.812  3.005
CDC WARRANT#                                        1.024 -0.833  2.883
MONETARY INSTRUMENT ABUSE                           1.020 -0.838  2.930
AGG. SECOND DEGREE BATTERY                          1.019 -0.950  2.819
OPERATING VEHICLE WHILE INTOXICATED                 1.019 -0.948  2.883
ATTEMPT -  ENTRY-BUSINESS                           1.019 -0.841  2.934
MACHINE GUNS - UNLAWFUL ACTS                        1.018 -0.881  2.836
OBSCENE LIVE CONDUCT                                1.017 -0.873  2.858
ILLEGALLY SUPPLYING A FELON WITH A FIREARM          1.017 -0.886  2.907
PROSTITUTION                                        1.017 -0.782

In [17]:
var_names = list(trace.posterior.data_vars.keys())
print(var_names)

['b', 'b_race', 'intercept', 'charge_effect', 'rtcc_est', 'sd_intercept', 'sigma']


In [18]:
az.summary(trace.posterior)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
b,0.578,0.036,0.512,0.646,0.004,0.003,94.0,217.0,1.06
b_race,-0.002,1.026,-1.891,1.905,0.007,0.014,19273.0,5471.0,1.00
intercept[0],-2.266,1.533,-5.208,0.518,0.028,0.020,2906.0,4882.0,1.00
intercept[1],0.493,1.004,-1.365,2.381,0.015,0.012,4782.0,5099.0,1.00
intercept[2],-0.566,1.196,-2.863,1.543,0.026,0.018,2140.0,4346.0,1.00
...,...,...,...,...,...,...,...,...,...
rtcc_est[509],7.522,2.509,3.000,12.000,0.107,0.076,550.0,767.0,1.01
rtcc_est[510],0.312,0.649,0.000,1.000,0.028,0.020,540.0,575.0,1.01
rtcc_est[511],6.195,2.223,3.000,10.000,0.084,0.060,681.0,926.0,1.01
sd_intercept,2.452,0.174,2.121,2.767,0.006,0.004,972.0,4423.0,1.01


In [19]:
charge_summary = az.summary(trace.posterior["charge_effect"], hdi_prob=0.95, round_to=2, var_names=['charge_effect'])
charge_summary = pd.DataFrame(charge_summary)
charge_means = charge_summary.mean()
print(charge_means)

race_summary = az.summary(trace.posterior["b"], hdi_prob=0.95, round_to=2, var_names=['b'])
race_summary = pd.DataFrame(race_summary)
race_means = race_summary.mean()
print(race_means)

mean             0.745709
sd               0.936873
hdi_2.5%        -1.079782
hdi_97.5%        2.576327
mcse_mean        0.010000
mcse_sd          0.010000
ess_bulk     15887.151927
ess_tail      5713.632145
r_hat            1.000000
dtype: float64
mean           0.58
sd             0.04
hdi_2.5%       0.51
hdi_97.5%      0.65
mcse_mean      0.00
mcse_sd        0.00
ess_bulk      94.48
ess_tail     216.61
r_hat          1.06
dtype: float64


In [20]:
# sns.kdeplot(ppc_white.posterior_predictive['rtcc'].values.reshape(-1, 2), label='White')
# sns.kdeplot(ppc_black.posterior_predictive['rtcc'].values.reshape(-1, 2), label='Black')
# plt.xlabel('Number of RTCC Requests')
# plt.ylabel('Density')
# plt.title('Posterior Predictive Check for "ARMED ROBBERY"')
# plt.legend()
# plt.show()